First things first. Let us set up the environment with the requried packages for this notebook:

In [335]:
for p in ("Knet", "Plots", "Plotly.jl", "DataFrames")
    Pkg.installed(p) == nothing && Pkg.add(p)
end

using Knet, Plots, DataFrames
gr()

Plots.GRBackend()

In this notebook we introduce the following Julia/Knet packages and functions:

* Julia's package [DataFrames](https://en.wikibooks.org/wiki/Introducing_Julia/DataFrames)
* Julia's (technically `JuliaDB`) function [loadtable](http://juliadb.org/latest/api/io.html#JuliaDB.loadtable)
* Knet's function [minibach](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.minibatch)


# Binary classification with logistic regression

In the last tutorial we worked through how to implement a [linear regression model](https://github.com/moralesq/Knet-the-Julia-dope/blob/master/chapter02_supervised-learning/linear-regression.ipynb).

Regression is the hammer we reach for when we want to answer *how much?* or *how many?* questions. If you want to predict the number price at which a house will be sold, or the number of wins a baseball team might have,  or the number of days that a patient will remain hospitalized before being discharged, then you're probably looking for a regression model.

Based on our experience, in industry, we're more often interested in making categorical assignments. *Does this email belong in the spam folder or the inbox*? *How likely is this custromer to sign up for subscription service?* When we're interested in either assigning datapoints to categories or assessing the *probability* that a category applies, we call this task *classification*. 

The simplest kind of classification problem is *binary classification*, when there are only two categories, so let's start there. Let's call our two categories the positive class $y_i=1$ and the negative class $y_i = 0$ (another common way of defining the labels are $y_i=\pm1$). Even with just two categories, and even confining ourselves to linear models, 
there are many ways we might approach the problem. For example, we might try to draw a line that best separates the points:

![](../img/linear-separator.png)

A whole family of algorithms called support vector machines pursue this approach.
The main idea here is choose a line that maximizes the marigin to the closest data points on either side of the decision boundary.  In these appraoches, only the points closest to the decision boundary (the support vectors)  actually influence the choice of the linear separator.

With neural networks, we usually appraoch the problem differently. Instead of just trying to separate the points, we train *probabilistic classifiers* which estimate, for each data point $\boldsymbol{x}$, the *conditional probability* $\mathbb{P}(y|\boldsymbol{x})$ that it belongs to class $y$. 

Recall that in linear regression, we made predictions of the form

$$ \hat{y} = \boldsymbol{w}^T \boldsymbol{x} + b, $$

where $\hat{y},b\in\mathbb{R}$ and $\boldsymbol{w},\boldsymbol{x}\in\mathbb{R}^d$. We are interested in asking the question *"what is the probability that example $\boldsymbol{x}$ belongs to the positive class?"* A regular linear model is a poor choice here because it can output values greater than $1$ or less than $0$. To coerce reasonable answers from our model,  we're going to modify it slightly, by running the linear function through a sigmoid activation function $\sigma$:

$$ \hat{y} =\sigma(\boldsymbol{w}^T \boldsymbol{x} + b). $$

The sigmoid function $\sigma$, sometimes called a squashing function or a *logistic* function - thus the name logistic regression - maps a real-valued input to the range 0 to 1. Indeed, the logistic function $\sigma(z)$ is a good choice since it has the form of a probability, i.e. $\sigma(-z)=1-\sigma(z)$ and $\sigma(z)\in (0,1)$ as $z\rightarrow \pm \infty$. If we pick the labels $y\in(0,1)$ we may assign  

\begin{equation}
\begin{aligned}
\mathbb{P}(y=1|z) & =\sigma(z)=\frac{1}{1+e^{-z}}\\
\mathbb{P}(y=0|z) & =1-\sigma(z)=\frac{1}{1+e^{z}}\\
\end{aligned}
\end{equation}


which can be written more compactly as $\mathbb{P}(y|z)  =\sigma(z)^y(1-\sigma(z))^{1-y}$. Let us define and visualize this function:

In [336]:
σ(z) = 1 ./ (1 + exp.(-z))
plot(-5:0.1:5, σ(-5:0.1:5), xlabel=:z, ylabel="σ(z)", title="Logistic Function", legend=false, size=(400,200))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 Logistic Function 
 
 
 z 
 
 
 ?(z)

Note that and input of $0$ gives a value of $.5$. 
So in the common case, where we want to predict positive whenever the probability is greater than $.5$
and negative whenever the probability is less than $.5$,
we can just look at the sign of $\boldsymbol{w}^T \boldsymbol{x} + b$. Formally, in (binary) classification problems one aims at finding a classification rule (also called decision rule) which is a binary valued function on the input sapce $c:\mathcal{X}\rightarrow\{0,1\}$ (in this example $\mathcal{X}=\mathbb{R}^d$). However, direct minimization of the classification error is not computationally feasible mostly because the classification loss is not convex (to be discussed later). In practice, once looks for real valued (rather than binary valued) function $f:\mathcal{X}\rightarrow \mathbb{R}$ and replaces the loss function with some  convex loss. A classification rule is then obtained by taking the sign. 

## Binary cross-entropy loss

Now that we've got a model that outputs probabilities,
we need to choose a loss function.
When we wanted to predict *how much* we used squared error $y-\hat{y}^2$,
as our measure our model's performance. 

Since now we're thinking about outputing probabilities,
one natural objective is to say that we should choose the weights (or parameters $\theta$ )
that give the actual labels in the training data highest probability. For $n$ samples $\{x_i,y_i\}$ we want to maximize

$$\max_{\theta} \mathbb{P}_{\theta}\big( y_1,\dots,y_n \big|\,\boldsymbol{x}_1,\dots\boldsymbol{x}_n \big)$$

Because each example is independent of the others, and each label depends only on the features of the corresponding examples, we can rewirte the above as

$$\max_\theta \prod_i^n\mathbb{P}_\theta(y_i| \boldsymbol{x}_i)=\max_{\theta} \mathbb{P}_{\theta}\big(y_1|\boldsymbol{x}_1\big)\mathbb{P}_{\theta}\big(y_2|\boldsymbol{x}_2\big)\cdots\mathbb{P}_{\theta}\big(y_n|\boldsymbol{x}_n\big)$$

This function is a product over the examples, but in general, because we want to train by stochastic gradient descent, it's a lot easier to work with a loss function that breaks down as a sum over the training examples. 

$$\max_\theta \log\big(\prod_i^n\mathbb{P}(y_i|\boldsymbol{x}_i)\big)= \sum_i^m\log\big(\mathbb{P}(y_i|\boldsymbol{x}_i)\big)=\log\big(\mathbb{P}(y_1|\boldsymbol{x}_1)\big)+\cdots+\log\big(\mathbb{P}(y_n|\boldsymbol{x}_n)\big)$$

Because we typically express our objective as a *loss* we can just flip the sign, giving us the *negative log probability:*

$$  \min_\theta \Big(- \sum_i^m\log\big(\mathbb{P}(y_i|\boldsymbol{x}_i)\big)\Big)$$

Recall that we can write $\mathbb{P}_\theta(y_1|z_i)$ compactly as

$$\mathbb{P}_\theta(y_i|z_i) =\sigma(z_i)^{y_i}(1-\sigma(z_i))^{1-y_i},$$

where $\hat{y}_i = \sigma(z_i) = \sigma(\boldsymbol{w}^T \boldsymbol{x}_i + b)$ and $\theta=\{w,b\}$ are the parameters to be optimized. Let us work through this expresion. With the (important) relation $\sigma(-z) = 1-\sigma(z)$ we have

\begin{equation}
\begin{aligned}
\log\big(\mathbb{P}_\theta(y|z)\big)&=
\log\big(\sigma(z)^{y}(1-\sigma(z))^{1-y}\big)\\
&=y\log\sigma(z) + (1-y)\log(1-\sigma(z))\\
&=y\big(\log\sigma(z)-\log\sigma(-z)\big) + \log\sigma(-z)\\
&=y\log \frac{\sigma(z)}{\sigma(-z)} + \log\sigma(-z)\\
&=y\log\Big( \frac{1+e^{z}}{1+e^{-z}} \Big) + \log\sigma(-z)\\
&=y\log\Big( \frac{e^{z}(e^{-z}+1)}{1+e^{-z}} \Big) + \log\sigma(-z)\\
&=yz + \log\sigma(-z)
\end{aligned}
\end{equation}

Therefore we take the negative of this expression and minimize the objective function 

$$l = \sum_{i=1}^n y_iz_i + \log(1+e^z)$$

whew! what a bunch math! If you're learning machine learning for the first time, that might have been too much information too quickly. Let's take a look at this loss function and break down what's going on more slowly.

We started with the espression 

$$\mathbb{P}_\theta(y_i|z_i) =\sigma(z_i)^{y_i}(1-\sigma(z_i))^{1-y_i},$$

where $\hat{y}_i = \sigma(z_i) = \sigma(\boldsymbol{w}^T \boldsymbol{x}_i + b)$. This is the conditional probability. 

We then found that the loss function depended on two terms:

* $y_i\log \hat{y}_i$
* $(1-y_i)\log (1-\hat{y}_i)$

But recall that we are intepreting $\hat{y}_i=\sigma(z_i)$ as a probability that $x_i$ has a given label, namely $\mathbb{P}(y_i=1|z_i)=\sigma(z_i)$ and $\mathbb{P}(y_i=0|z_i)=1-\sigma(z_i)$. Because $y_i$ only takes values $0$ or $1$, for an given data point, one of these terms disapears. 
When $y_i$ is $1$, this loss says that we should maximize $\log \hat{y}_i$, giving higher probability to the *correct* answer. 
When $y_i$ is $0$, this loss function takes value $\log (1-\hat{y}_i)$. That says that we should maximize the value $1-\hat{y}$ which we already know is the probability assigned to $\boldsymbol{x}_i$ belonging to the negative class.


Note that this loss function is commonly called *log loss* and also commonly referred to as *binary cross entropy*. It is a special case of negative log [likelihood](https://en.wikipedia.org/wiki/Likelihood_function). And it is a special case of [cross-entropy](https://en.wikipedia.org/wiki/Cross_entropy), which can apply to the multi-class ($>2$) setting. 

**If instead we were to use the labels $y_i=\pm1$, the loss function has to modified to $\log(1+e^{-z})$. This usually leads to a lot of confussion as to why there exists two versions of logistic regression. See [here](https://stats.stackexchange.com/questions/250937/which-loss-function-is-correct-for-logistic-regression/279698#279698) for more information on the topic**

## Data

As usual, we'll want to work out these concepts using a real dataset. We will use the Medical Appointment No Shows [dataset](https://www.kaggle.com/joniarroba/noshowappointments). For a different example with a different dataset please visit the [original](https://github.com/zackchase/mxnet-the-straight-dope/blob/master/chapter02_supervised-learning/logistic-regression-gluon.ipynb) mxnet version. The medical appoinments raw dataset consists of 13 features and 1 binary label. The labels are show or no show to an scheduled medical appoinment, and the features include gender, age, neighbourhood, etc. 

We will use Julia's pakage `DataFrames` and the type `DataFrame` to load and manipulate the data. Let us load the data and explore the type and content of the output:

In [337]:
rawdata = readtable("../datasets/KaggleV2-May-2016.csv");
typeof(rawdata)

DataFrames.DataFrame

In [338]:
rawdata[1:5, :]

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No_show
1,2.9872499824296e13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
2,5.58997776694438e14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
3,4.262962299951e12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
4,8.67951213174e11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
5,8.841186448183e12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


As you can see, `rawdata` is of type `DataFrame` and it automatically reads the csv file into a Frame. By default it assumes that the names of each column are on the firstrow. There are MANY ways of manipulating this data and you should [become familiar](https://en.wikibooks.org/wiki/Introducing_Julia/DataFrames) with them. For example, we can access one particular feature with its name:

In [339]:
rawdata[1:5, :Gender], rawdata[1:5, :No_show]

(String["F", "M", "F", "F", "F"], String["No", "No", "No", "No", "No"])

Our first obstacle is to decide how to encode some of the features. For example, there are multiple ways to [encode the month](https://stats.stackexchange.com/questions/108977/whats-the-optimal-way-to-encode-a-month-feature). We could encode a single variable with 12 discrete values {Month = 1, 2, ..., 12} or equivalent. This might be OK if you really want to treat each month separately. However, there is a potential problem of modulo distance, i.e. if you use numbers 1, ... , 12 then each month differs by 1 from the previous month, except for Dec=12 & Jan=1. Let us take a look at this feature to see what we're dealing with. 

First we define the feature encoding functions and we split the date into the relevant components:

In [340]:
gender(x)            = 1.0(x .== "M") # one for male 
appointment_month(x) = parse(Int64, x[2]);
appointment_date(x)  = parse(Int64, split(x[3], 'T')[1]);
appointment_hour(x)  = parse(Int64, split(split(x[3], 'T')[2], ':')[1]);

In [341]:
s = split.(rawdata[:, :ScheduledDay], "-");

syear  = appointment_year.(s);
smonth = appointment_month.(s)
sdate  = appointment_date.(s);
shour  = appointment_hour.(s);

LoadError: [91mUndefVarError: appointment_year not defined[39m

Let us redefine the labels from `:No_show=[yes, no]` to `:No_show=[1, 0]` such that a subject with a label of `1` did not show up to the appoinment.  

In [342]:
rawdata[:, :No_show] = map(Float64, rawdata[:, :No_show] .== "Yes");

We can now plot the conditional probabilities for the hourn and month for all subjects:

In [343]:
p1 = histogram(shour[rawdata[:, :No_show] .== 1],  bins=0.5:1:23.5, norm=true, label="y=No_show", title="P(y|hour)")
     histogram!(shour[rawdata[:, :No_show] .== 0], bins=0.5:1:23.5, norm=true, label="y=show", title="P(y|hour)");

p2 = histogram(smonth[rawdata[:, :No_show] .== 1],  bins=0.5:11.5, norm=true, label="y=No_show", title="P(y|month)")
     histogram!(smonth[rawdata[:, :No_show] .== 0], bins=0.5:11.5, norm=true, label="y=show", title="P(y|month)");

In [344]:
plot(p1,p2, size=(700,300))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 P(y|hour) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y=No_show 
 
 
 
 
 y=show 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6 
 
 
 P(y|month) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y=No_show 
 
 
 
 
 y=show

As you can see, we don't have any data after June, so for now it's safe to simply encode the month like this. In addition, you can verify that almost the entire dataset has the same year (2016) and therefore for simplicity we will ignore it. We have a similar situation with the hour since most appoinments were done early during the day and therefore we can take the same approach. 

Now, let's examine what these plots are saying. On the left we have the conditional probability $\mathbb{P}(y|hour)$, i.e. what is the probability that the subject will show up given the time of the day *in which the appoinment was scheduled* (discretized in hours). The results are quite interesting! this plot says that it's more probable that the subject will show up if the appoinment was scheduled is at before 9am (peaking at 7am), and more probable that it will be a no show if the appoinment was scheduled after 9am (peaking at 2pm). ummm, perhaps early birds are more reliable? 

On the right we have $\mathbb{P}(y|month)$. In this case we find that a subject is more probable to show up if the appoinment  was scheduled after summer (peaking in May), and more likely to be a no show before that. Perhaps those that schedule early during the year assume they will be able to re-schedule at a later month? or is it a New Year's resolution syndrome?

ok, time to get serious. Let us pre-process this datast now. Notice that

* The year will be ignored since it does no vary 
* The appoinment hour is ignored since it is not shown
* For simplicity we will ingnore the Neighbourhood field 

In [345]:
smonth = appointment_month.(split.(rawdata[:, :ScheduledDay], "-"););
sdate  = appointment_date.(split.(rawdata[:, :ScheduledDay], "-"););
shour  = appointment_hour.(split.(rawdata[:, :ScheduledDay], "-"););
amonth = appointment_month.(split.(rawdata[:, :AppointmentDay], "-"););
adate  = appointment_date.(split.(rawdata[:, :AppointmentDay], "-"););

In [346]:
dict = Dict("Jan"  => 1, "Feb" => 2, "Mar"  => 3,"Apr" => 4,  "May" => 5, "June" => 6,
            "July" => 7, "Aug" => 8, "Sept" => 9,"Oct" => 10, "Nov" => 11, "Dec" => 12);

In [347]:
x = DataFrame()
for (key, val) in dict
    skey = @sprintf "s%s" key
    akey = @sprintf "a%s" key
    y1 = 1.0(smonth .== val)
    y2 = 1.0(amonth .== val)
    if sum(y1) > 0
        x[:, [Symbol(skey)]] = y1
    end
    if sum(y2) > 0
        x[:, [Symbol(akey)]] = y2
    end
end

for date =1:31
    skey = @sprintf "sdate_%d" date
    akey = @sprintf "adate_%d" date
    y1 = 1.0(sdate .== date)
    y2 = 1.0(adate .== date)
    
    if sum(y1) > 0
        x[:, [Symbol(skey)]] = y1
    end
    if sum(y2) > 0
        x[:, [Symbol(akey)]] = y2
    end
end

for hour = 1:24
    skey = @sprintf "shour%d" hour
    y = 1.0(shour .== hour)
    if sum(y) > 0
        x[:, [Symbol(skey)]] = y
    end
end

x[:, :age]    = (rawdata[:, :Age]- minimum(rawdata[:, :Age])) / (maximum(rawdata[:, :Age]) - minimum(rawdata[:, :Age]))
x[:, :Scholarship]  = rawdata[:, :Scholarship]
x[:, :Hipertension] = rawdata[:, :Hipertension]
x[:, :Diabetes]     = rawdata[:, :Diabetes]
x[:, :Alcoholism]   = rawdata[:, :Alcoholism]
x[:, :Handcap]      = rawdata[:, :Handcap]
x[:, :SMS_received] = rawdata[:, :SMS_received];
x[:, :label]        = rawdata[:, :No_show];

Now, let us split the data into training and testing batches and use Knet's [minibatch](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.minibatch) function to as a data provider:

In [348]:

test = 0.10;
n    = round(Int, (1 - test) * size(x, 1));
r    = randperm(size(x, 1))
xtrn = convert(Array{Float32, 2}, Array(x[r[1:n], 1:end-1])');
ytrn = convert(Array{UInt8, 1},   Array(x[r[1:n], end]));
xtst = convert(Array{Float32, 2}, Array(x[r[n+1:end], 1:end-1])');
ytst = convert(Array{UInt8, 1},   Array(x[r[n+1:end], end]));

In [349]:
Atype = Array{Float32, 2}
dtrn = minibatch(xtrn, ytrn, 50; shuffle=true);
dtst = minibatch(xtst, ytst, 50; shuffle=true);

Notice that we tranposed `xtrn` and `ytrn` since `minibatch` assumes the samples are on the last column. Let's take a look at the data type of `dtrn`:

We are now ready to start building our model:

In [350]:
pred(w, x) = w[1] * x .+ w[2];

function loss(w, x, y)
    yhat = sigm.(pred(w, x))
    return -sum(y .* log.(yhat) + (1-y) .* log.(1-yhat))
end

lossgradient  = grad(loss)

(::gradfun) (generic function with 1 method)

In [351]:
function train(w, data; lr=1e-6, epochs=5)
    for epoch = 1:epochs
        closs = 0
        for (x,y) in data
            closs += loss(w,x,y)
            g = lossgradient(w, x, y)
            for i = 1:length(w)
                w[i] -= lr * g[i]
            end
        end
        
        print(closs/length(data), "\n")
    end
end

train (generic function with 1 method)

In [352]:
w = map(atype, [randn(1, size(xtrn, 1)), zeros(Float32,1,1) ]);

In [353]:
sum((sign.(pred(w, xtrn)) .+ 1) / 2 .== ytrn') / size(ytrn, 1),
sum((sign.(pred(w, xtst)) .+ 1) / 2 .== ytst') / size(ytst, 1)

(0.5069264330377787, 0.512078168822944)

In [354]:
train(w,dtrn)

1994.4585303918188
1739.8089964047092
1696.0446875588154
1665.061836302719
1638.4436695089912


In [355]:
sum((sign.(pred(w, xtrn)) .+ 1) / 2 .== ytrn') / size(ytrn, 1),
sum((sign.(pred(w, xtst)) .+ 1) / 2 .== ytst') / size(ytst, 1)

(0.746024086696021, 0.7446847009861576)

## The Knet way

[nll](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.nll)  computes the negative log likelihood of your predictions compared to the correct answers.

In [204]:
pred(w, x)    = w[1] * x .+ w[2];
loss(w, x, y) = nll(pred(w,x), y)
lossgradient  = grad(loss)

(::gradfun) (generic function with 1 method)

In [206]:
w = map(atype, [0.1f0*randn(2, size(xtrn, 1)), zeros(Float32, 1, 1) ]);
accuracy(pred(w, xtrn), ytrn), accuracy(pred(w, xtst), ytst)

(0.5719685545971812, 0.517235139781055)

In [207]:
train(w, dtrn)

0.6025546453683817
0.536764427490172
0.5179977070104435
0.5104632364979124
0.5063369927104686


In [208]:
accuracy(pred(w, xtrn), ytrn), accuracy(pred(w, xtst), ytst)

(0.7913927257373786, 0.7960734642178594)